In [1]:
num_list = [1, 2, 3, 4, 5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

`[1, 2, 3, 4, 5]` はリストではなく、配列(もしくはベクトル)である。

In [2]:
num_arr = [1, 2, 3, 4, 5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [3]:
function f(num_arr)
    sum = 0
    n_iters = length(num_arr)
    for i in 1:n_iters
        sum += num_arr[i]
    end
    return sum
end

f(num_arr)

15

`f(num_arr)` は配列(Array)以外にも適用可能だが `num_arr` という名前でいいのか？

In [4]:
f(1:5)

15

In [5]:
[typeof(1:5); [∘(fill(supertype, k)...)(typeof(1:5)) for k in 1:5]]

6-element Vector{DataType}:
 UnitRange{Int64}
 AbstractUnitRange{Int64}
 OrdinalRange{Int64, Int64}
 AbstractRange{Int64}
 AbstractVector{Int64} (alias for AbstractArray{Int64, 1})
 Any

あと、`Base.sum` とローカル変数の名前が被っていても確かに問題ないが、それでもいいのか？

In [6]:
function f_rev1(num_abst_arr)
    total = 0
    n_iters = length(num_abst_arr)
    for i in 1:n_iters
        total += num_arr[i]
    end
    return total
end

f_rev1(num_arr)

15

`f_rev1([k for k in 1:5])` は計算できるが、`f_rev1(k for k in 1:5)` はエラーになる。

In [7]:
f_rev1([k for k in 1:5])

15

In [8]:
f_rev1(k for k in 1:5)

15

In [9]:
G = (k for k in 1:5)

Base.Generator{UnitRange{Int64}, typeof(identity)}(identity, 1:5)

In [10]:
function f_rev2(num_iter)
    total = 0
    for num in num_iter
        total += num
    end
    return total
end

f_rev2(num_arr)

15

In [11]:
f_rev2(k for k in 1:5)

15

引数 `num_iter` の要素が整数でない場合に型不安定！

In [12]:
@code_warntype f_rev2([1.0, 2.0, 3.0, 4.0, 5.0])

MethodInstance for f_rev2(::Vector{Float64})
  from f_rev2(num_iter) in Main at In[10]:1
Arguments
  #self#::Core.Const(f_rev2)
  num_iter::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  total::Union{Float64, Int64}
  num::Float64
Body::Union{Float64, Int64}
1 ─       (total = 0)
│   %2  = num_iter::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}
│         (num = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (total = total + num)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return total



In [13]:
function f_rev3(num_iter)
    total = zero(first(num_iter))
    for num in num_iter
        total += num
    end
    return total
end

f_rev3(num_arr)

15

In [14]:
f_rev3(k for k in 1:5)

15

In [15]:
@code_warntype f_rev3([1.0, 2.0, 3.0, 4.0, 5.0])

MethodInstance for f_rev3(::Vector{Float64})
  from f_rev3(num_iter) in Main at In[13]:1
Arguments
  #self#::Core.Const(f_rev3)
  num_iter::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  total::Float64
  num::Float64
Body::Float64
1 ─ %1  = Main.first(num_iter)::Float64
│         (total = Main.zero(%1))
│   %3  = num_iter::Vector{Float64}
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_3::Tuple{Float64, Int64}
│         (num = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│         (total = total + num)
│         (@_3 = Base.iterate(%3, %10))
│   %13 = (@_3 === nothing)::Bool
│   %14 = Base.not_int(%13)::Bool
└──       goto #4 if not %14
3 ─       goto #2
4 ┄       return total



`f_rev3(num_iter)` は引数 `num_iter` の要素が数でなくても使えるのに、`num_iter` という名前でいいのか？

In [16]:
f_rev3([[1, 2], [3, 4], [5, 6]])

2-element Vector{Int64}:
  9
 12

In [17]:
function f_rev4(iter)
    total = zero(first(iter))
    for val in iter
        total += val
    end
    return total
end

f_rev4(num_arr)

15

In [18]:
f_rev4(k for k in 1:5)

15

In [19]:
f_rev4([[1, 2], [3, 4], [5, 6]])

2-element Vector{Int64}:
  9
 12

Juliaでは全ての文が値を持つ式なので函数の終わりでの `return` は省略できる。

In [20]:
function f_rev5(iter)
    total = zero(first(iter))
    for val in iter
        total += val
    end
    total
end

f_rev5(num_arr)

15

In [21]:
f_rev5(k for k in 1:5)

15

In [22]:
f_rev5([[1, 2], [3, 4], [5, 6]])

2-element Vector{Int64}:
  9
 12

`iter` が空の場合はエラーになる。

In [23]:
zeros(0)

Float64[]

In [24]:
f_rev5(zeros(0))

LoadError: BoundsError: attempt to access 0-element Vector{Float64} at index [1]

In [25]:
function f_rev6(iter, init = isempty(iter) ? zero(eltype(iter)) : zero(first(iter)))
    total = init
    for val in iter
        total += val
    end
    total
end

f_rev6(num_arr)

15

In [26]:
f_rev6(k for k in 1:5)

15

In [27]:
f_rev6([[1, 2], [3, 4], [5, 6]])

2-element Vector{Int64}:
  9
 12

In [28]:
f_rev6(zeros())

0.0

In [29]:
f_rev6(Vector{Int64}[])

LoadError: MethodError: no method matching zero(::Type{Vector{Int64}})
[0mClosest candidates are:
[0m  zero([91m::Union{Type{P}, P}[39m) where P<:Dates.Period at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.7\Dates\src\periods.jl:53
[0m  zero([91m::SparseArrays.AbstractSparseArray[39m) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.7\SparseArrays\src\SparseArrays.jl:55
[0m  zero([91m::AbstractArray{T}[39m) where T at abstractarray.jl:1136
[0m  ...

In [30]:
f_rev6(Vector{Int64}[], [0, 0])

2-element Vector{Int64}:
 0
 0

函数の定義がこれだけ短いなら、「意味のある変数名」ではなく、1文字変数名でもいいんじゃね？

In [31]:
function f_rev7(A, o = isempty(A) ? zero(eltype(A)) : zero(first(A)))
    s = o
    for v in A
        s += v
    end
    s
end

f_rev7(num_arr)

15

In [32]:
f_rev7(k for k in 1:5)

15

In [33]:
f_rev7([[1, 2], [3, 4], [5, 6]])

2-element Vector{Int64}:
  9
 12

In [34]:
f_rev7(zeros())

0.0

In [35]:
f_rev7(Vector{Int64}[], [0, 0])

2-element Vector{Int64}:
 0
 0

In [36]:
A = [
    1 2
    3 4
]
f_rev7(A)

10

Juliaの配列のインデックスは任意始まりにできる。

In [37]:
using OffsetArrays
a = OffsetArray([k^3 for k in -3:3], -3:3)

7-element OffsetArray(::Vector{Int64}, -3:3) with eltype Int64 with indices -3:3:
 -27
  -8
  -1
   0
   1
   8
  27

In [38]:
@show collect(eachindex(a))

for i in eachindex(a)
    println("a[", i, "] = ", a[i])
end

collect(eachindex(a)) = [-3, -2, -1, 0, 1, 2, 3]
a[-3] = -27
a[-2] = -8
a[-1] = -1
a[0] = 0
a[1] = 1
a[2] = 8
a[3] = 27


In [39]:
@show pairs(a)
for (i, v) in pairs(a)
    println("a[", i, "] = ", v)
end

pairs(a) = Base.Pairs(-3 => -27, -2 => -8, -1 => -1, 0 => 0, 1 => 1, 2 => 8, 3 => 27)
a[-3] = -27
a[-2] = -8
a[-1] = -1
a[0] = 0
a[1] = 1
a[2] = 8
a[3] = 27


In [40]:
pairs(a)

pairs(::OffsetVector{Int64, Vector{Int64}})(...):
  -3 => -27
  -2 => -8
  -1 => -1
  0  => 0
  1  => 1
  2  => 8
  3  => 27

In [41]:
A = OffsetArray(reshape(1:24, 4, 3, 2), -1:2, -1:1, 0:1)

4×3×2 OffsetArray(reshape(::UnitRange{Int64}, 4, 3, 2), -1:2, -1:1, 0:1) with eltype Int64 with indices -1:2×-1:1×0:1:
[:, :, 0] =
 1  5   9
 2  6  10
 3  7  11
 4  8  12

[:, :, 1] =
 13  17  21
 14  18  22
 15  19  23
 16  20  24

In [42]:
for k in axes(A, 3), j in axes(A, 2), i in axes(A, 1)
    println("A[", i, ", ", j, ", ", k, "] = ", A[i, j, k])
end

A[-1, -1, 0] = 1
A[0, -1, 0] = 2
A[1, -1, 0] = 3
A[2, -1, 0] = 4
A[-1, 0, 0] = 5
A[0, 0, 0] = 6
A[1, 0, 0] = 7
A[2, 0, 0] = 8
A[-1, 1, 0] = 9
A[0, 1, 0] = 10
A[1, 1, 0] = 11
A[2, 1, 0] = 12
A[-1, -1, 1] = 13
A[0, -1, 1] = 14
A[1, -1, 1] = 15
A[2, -1, 1] = 16
A[-1, 0, 1] = 17
A[0, 0, 1] = 18
A[1, 0, 1] = 19
A[2, 0, 1] = 20
A[-1, 1, 1] = 21
A[0, 1, 1] = 22
A[1, 1, 1] = 23
A[2, 1, 1] = 24
